# In this notebook:
- Systematically remove scenes:  >2% unobserved pixels
- Manually remove scenes: visual inspection of duplicate dates
- Create a csv with PS filenames for corresponding ASO filenames


Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import geopandas as gpd
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import rioxarray as rxr
from datetime import datetime
import glob
import os
import contextily as ctx
import xarray as xr
from collections import Counter
import cmcrameri.cm as cmc
import shutil

from working_sca_funcs import *

In [2]:
# Define year you are working on
year = '2022'

# Define basin you are working on
# name = 'DLNY'
# name = 'LYMC'
# name = 'LYTB'
# name = 'H120'
# name = 'DNBC'
# name = 'BUDD'
name = 'COPP'

# model = 'V1' # Justin's manual classification 2023 DLNY
# model = 'V2' # 2023 DLNY ASO 2 class
# model = 'V3' # 2023 DLNY ASO 4 class
# model = 'V4' # 2023 DLNY ASO 2 class NDVI
model = 'base' # 2022 BUDD ASO 2 class
# model = 'V6' # 2022 BUDD manual classification snow/no snow/artifact all trained on same images
#model = 'V7' # 2022 BUDD manual classification snow/no snow trained on set of images and artifacts on different images
# model = 'V9'
# raw PS images
ps_raw = f'/data0/images/planet/emma/planet/{name}/'
ps_raw_subdir = sorted([d for d in glob.glob(ps_raw + str(year) + '*') if os.path.isdir(d)])

# SCA directory
#ps_sca_dir = f'/data0/images/planet/emma/planet/processed_SCA/{name}/{model}/'
ps_sca_dir = f'/home/etboud/projects/data/temp/{name}/{model}/'
ps_sca_tif = glob.glob(ps_sca_dir + f'*{year}*.tif')

# ASO directory
aso_dir = f'/home/etboud/projects/data/aso/validation/{name}/V2/'
aso_tif = glob.glob(os.path.join(aso_dir, f'*{year}*3m*.tif')) 

# Create basin shapefile
BS = f'/home/etboud/projects/data/basins/{name}/{name}_4326.geojson'
basin = gpd.read_file(BS)
basin = basin.to_crs('EPSG:32611') 

# Extract dates from file names
ps_dates = extract_dates([], ps_sca_tif)[1]  
aso_dates = extract_dates(aso_tif, [])[0]

# canopy height model
CHM = '/home/etboud/projects/data/CHM/USCATB20140827_chm_3p0m.tif'

### Systematically remove bad scenes
Anything with unobs >.02 will be removed
Run this when the model is set to V1

In [ ]:
chm_mask,mean,max,fcan = create_binary_chm(CHM, basin)
df = create_ps_df(ps_sca_tif, basin, name, chm_mask)

In [ ]:
# Visualize processed maps
for file in ps_sca_tif:
    if os.path.basename(file).split(".")[0] not in df['fn'].values:
        sca = rxr.open_rasterio(file, all_touched=False, drop=True, masked=True)
        sca.values = np.where(np.isnan(sca.values), 0, sca.values)
        sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
        sca = np.squeeze(sca.values)
        im = plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
        plt.title(os.path.basename(file))
        plt.colorbar(im)
        plt.show()

In [ ]:
# Moving files of poor quality to a different directory ( i beleive over 2% unobds)
path = f'/home/etboud/projects/data/planet/processed_SCA/{name}/{model}/'
move_to = f'/home/etboud/projects/data/planet/processed_SCA/{name}/{model}/not_good/'
for file in ps_sca_tif:
    if os.path.basename(file).split(".")[0] not in df['fn'].values:
        src = path+os.path.basename(file)
        dst = move_to+os.path.basename(file)
        shutil.move(src, dst)
        #print(file)

## Manually identify which scene are bad
Create a not_good folder for one model run and move bad scenes into it after running INSPECTION CELL below

In [ ]:
# INSPECTION CELL
# Run on one model directory to manually determine which file to move dor duplicate or triplicate dates
# Change len(files_for_date) between 2 or 3 to and axis to 2 or 3

ps_dates = extract_dates([], ps_sca_tif)[1]
# Extract dates from the list
dates = [item[0] for item in ps_dates]

# Count occurrences of each date
date_counter = Counter(dates)

# Find dates that occur more than once
duplicate_dates = [date for date, count in date_counter.items() if count > 1]

# Plot PS for each duplicate date
for date in duplicate_dates:
    print(f"Plotting data for date: {date}")
    files_for_date = [file_path for ps_date, file_path in ps_dates if ps_date == date]
    
    # Ensure there are exactly two files for this date
    if len(files_for_date) == 2:
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # Create two side-by-side plots
        
        for i, file_path in enumerate(files_for_date):
            fn = os.path.basename(file_path)
            # Load the data using rioxarray
            data = rxr.open_rasterio(file_path, all_touched=False, drop=True, masked=True)
            data.values = np.where(np.isnan(data.values), 0, data.values)
            data = data.rio.clip(basin.geometry.values, basin.crs,drop=True)
            # Plotting the first band
            data.isel(band=0).plot(ax=axes[i], cmap=cmc.batlow)
            axes[i].set_title(f"File: {fn}")
            axes[i].axis('off')  # Turn off axis
        
        plt.suptitle(f"Date: {date}")
        plt.tight_layout()
        plt.show()

In [ ]:
# Run for each Basin to create a list of PS scenes that are bad
# Run this when model is set to V1 (unobs scenes already in not_good model)
dir =  glob.glob(f'/data0/images/planet/emma/planet/processed_SCA/{name}/V5/not_good/'+'*.tif')
save_bad = []
for file in dir:
    if os.path.basename(file)[:4]==year:
        fn = os.path.basename(file)
        save_bad.append(fn)

In [ ]:
# Plot 'not_good' scenes
model = 'V5'
not_good = f'/data0/images/planet/emma/planet/processed_SCA/{name}/{model}/'
dir = glob.glob(not_good + '/*.tif')
for file in dir:
    fn = os.path.basename(file)
    date = fn.split('_')[0]
    id = fn.split('_')[1]
    code = f'{date}_{id}'
    

    
    for raw_file in ps_raw_subdir:
        if code in raw_file:
            matching_raw_file = glob.glob(raw_file+'/*/PSScene/*_SR_clip.tif')[0]
            rgb_image = rxr.open_rasterio(matching_raw_file,masked = True, drop=True)
            rgb_image = rgb_image.rio.clip(basin.geometry, basin.crs)
            _, _, _, _, rgb_image = calc_rgb(rgb_image)
            
            sca = rxr.open_rasterio(file, all_touched=False, drop=True, masked=True)
            sca.values = np.where(np.isnan(sca.values), 0, sca.values)
            sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
            sca = np.squeeze(sca.values)
            
            plt.subplot(1,2,1)
            plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
            plt.colorbar()
            plt.title(code)
            plt.axis('off')
            
            plt.subplot(1,2,2)
            plt.imshow(rgb_image)
            plt.axis('off')
            plt.show()
            

In [ ]:
# pull out the date and code from save bad

save_bad_codes = [fn.split('_')[0] + '_' + fn.split('_')[1] for fn in save_bad]


In [ ]:
#moving raw subdirectories for bad PS images to a different directory
path = f'/data0/images/planet/emma/planet/{name}/'
move_to = f'/data0/images/planet/emma/planet/{name}/not_good/'
dir = glob.glob(path + '/*.tif')
save_bad_codes = [fn.split('_')[0] + '_' + fn.split('_')[1] for fn in save_bad]

for file in ps_raw_subdir:
    fn = os.path.basename(file)
    date = fn.split('_')[0]
    id = fn.split('_')[1]
    code = f'{date}_{id}'
    subdir = file.split('/')[-1]
    if code in save_bad_codes:
        src = path+subdir
        dst = move_to+subdir
        shutil.move(src, dst)
        # print(subdir)

In [ ]:
# Move processed bad files to a new directory
other_model = 'V9'
path = f'/data0/images/planet/emma/planet/processed_SCA/{name}/{other_model}/'
move_to = f'/data0/images/planet/emma/planet/processed_SCA/{name}/{other_model}/not_good/'
dir = glob.glob(path + '/*.tif')
for file in dir:
    if os.path.basename(file) in save_bad:
        print(file)
        src = path+os.path.basename(file)
        dst = move_to+os.path.basename(file)
        shutil.move(src, dst)
        

In [ ]:
import glob
import os

# Get list of .tif files from both directories
dir_v5 = glob.glob('/data0/images/planet/emma/planet/processed_SCA/BUDD/V5/*.tif')
dir_v9 = glob.glob('/data0/images/planet/emma/planet/processed_SCA/BUDD/V9/*.tif')

# Get just the basenames (filename without the full path)
basenames_v5 = set(os.path.basename(f) for f in dir_v5)
basenames_v9 = set(os.path.basename(f) for f in dir_v9)


In [ ]:
# Find files that are in V5 but not in V9
files_in_v5_only = basenames_v5 - basenames_v9
files_in_v5_only

# Find files that are in V9 but not in V5
files_in_v9_only = basenames_v9 - basenames_v5
files_in_v9_only


In [ ]:
files_in_v5_only

In [ ]:
import netCDF4
import xarray as xr

In [ ]:
# file = '/home/etboud/projects/snow_mapping/Emma/BUDD_V5_2022_3_10_NDVI_QAQC.nc'
file = '/home/etboud/projects/snow_mapping/Emma/DLNY_V5_2023_10_10_NDVI_QAQC.nc'
dataset = xr.open_dataset(file)

In [ ]:
dataset

In [ ]:
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/BUDD/V5' + '/*2022*.tif')))
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/BUDD/V9' + '/*2022*.tif')))
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/BUDD/V7' + '/*2022*.tif')))
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/DLNY/V5' + '/*2023*.tif')))
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/DLNY/V9' + '/*2023*.tif')))
print(len(glob.glob('/data0/images/planet/emma/planet/processed_SCA/DLNY/V7' + '/*2023*.tif')))

In [ ]:
dir.sort()
for file in dir:
    if os.path.basename(file)[:4] == year:
        print(os.path.basename(file))
        

In [ ]:
# Visually check single file
#file = '/home/etboud/projects/data/planet/processed_SCA/LYMC/V1/20230628_174540_SCA.tif'
#file = '/home/etboud/projects/data/planet/processed_SCA/LYMC/V2/20230628_174540_SCA.tif'
# file = '/home/etboud/projects/data/planet/processed_SCA/LYMC/V2/20230622_182552_SCA.tif'
# sca = rioxarray.open_rasterio(file, all_touched=False, drop=True, masked=True)
# sca.values = np.where(np.isnan(sca.values), 0, sca.values)
# sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
# sca = np.squeeze(sca.values)
# im = plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
# plt.colorbar(im)
# plt.show()


# # Visually check aso validation tifs
# for file in val_tif:
#     aso = rioxarray.open_rasterio(file, all_touched=False, drop=True, masked=True)
#     aso.values = np.where(np.isnan(aso.values), 0, aso.values)
#     aso = aso.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
#     aso = np.squeeze(aso.values)
#     im = plt.imshow(aso,vmin= 0,vmax = 2, interpolation= 'none')
#     plt.colorbar(im)
#     plt.show()

for file in ps_sca_tif:
    sca = rioxarray.open_rasterio(file, all_touched=False, drop=True, masked=True)
    sca.values = np.where(np.isnan(sca.values), 0, sca.values)
    sca = sca.rio.clip(basin.geometry.values, crs=basin.crs, drop=True)
    sca = np.squeeze(sca.values)
    im = plt.imshow(sca,vmin= 0,vmax = 2, interpolation= 'none')
    plt.colorbar(im)
    plt.show()

### Save Corresponding Files in csv

In [3]:
def closest_dates(ps_dates, aso_date):
    # Extract dates from tuples
    ps_date_list = [date for date, _ in ps_dates]
    # Sort the ps_dates
    sorted_list = sorted(ps_date_list)
    
    closest_distance = None
    closest_dates = []

    for date in sorted_list:
        distance = abs((date - aso_date).days)
        
        if closest_distance is None or distance < closest_distance:
            closest_distance = distance
            closest_dates = [date]
        elif distance == closest_distance:
            closest_dates.append(date)

    return closest_dates

In [ ]:

# create dataframe to store all PS filenames with corresponding ASO flights
raw_fn = []
for aso_date, aso_file in sorted(aso_dates):
    closest = closest_dates(ps_dates, aso_date)

    closest_str = closest[0].strftime('%Y%m%d') # change index fromo to -1 to get the preceeding or following closest date when dates adjacent are equadistance
    # ps_dates = pd.to_datetime(closest_str, format='%Y-%m-%d') 
    print(closest_str)
    for fn in ps_sca_tif:
        if closest_str in fn:
            matching_model_files = glob.glob(fn)
            code = fn.split('/')[-1].split('_')[1]
            closest_code = f'{closest_str}_{code}'
    for file in ps_raw_subdir:
        if closest_code in file:
            matching_raw_files = glob.glob(file+'/*/PSScene/*_SR_clip.tif')[0]
            raw_fn.append({"date": closest_str, "raw_file": matching_raw_files,"model_file": matching_model_files, "aso_file": aso_file,"aso_date": aso_date})
            
pd_2020_0 = pd.DataFrame(raw_fn)

In [5]:
pd_2020_0

,date,raw_file,model_file,aso_file,aso_date
0,20220421,/data0/images/planet/emma/planet/COPP/20220421...,[/home/etboud/projects/data/temp/COPP/base/202...,/home/etboud/projects/data/aso/validation/COPP...,2022-04-21
1,20220515,/data0/images/planet/emma/planet/COPP/20220515...,[/home/etboud/projects/data/temp/COPP/base/202...,/home/etboud/projects/data/aso/validation/COPP...,2022-05-18


In [8]:
merged = pd_2020_0

In [ ]:
merged = pd.concat([pd_2022_0,pd_2022_1,pd_2023_0,pd_2023_1], ignore_index=True) #run in different cell after creating multiple DF for each year

In [9]:
def clean_string(x):
    if isinstance(x, list):
        # Convert list to string, then clean it
        x = ', '.join(map(str, x))
    elif not isinstance(x, str):
        return x  # Return as is if not a string or list

    # Now x is guaranteed to be a string, so we can clean it
    return x.replace('[', '').replace(']', '').replace("'", '')

# Apply the cleaning function to both columns
merged['raw_file'] = merged['raw_file'].apply(clean_string)
merged['model_file'] = merged['model_file'].apply(clean_string)

# Format date columns
merged['ps_date'] = pd.to_datetime(merged['date']).dt.strftime('%Y-%m-%d')
merged['aso_date'] = pd.to_datetime(merged['aso_date']).dt.strftime('%Y-%m-%d')

# Drop the original date column
merged.drop(columns=['date'], inplace=True)
merged_drop = merged.drop_duplicates()
#merged_drop.to_csv('corresponding_files.csv', index=False)


In [10]:
merged_drop.to_csv('COPP_corresponding_files.csv', index=False)

In [ ]:
existing = pd.read_csv('/home/etboud/projects/ps_sca/corresponding_files.csv')

In [ ]:
merged_drop

In [ ]:
existing

In [ ]:
pd.concat([existing, merged_drop], ignore_index=True).to_csv('corresponding_files_all.csv', index=False)

In [ ]:
# Loop through the corresponding files
for index, row in merged_drop.iterrows():
    raw_fn = row['raw_file']
    model_fn = row['model_file']
    aso_fn = row['aso_file']
    
    fn = raw_fn.split('/')[-1].split('_')
    id = f'{fn[0]}_{fn[1]}'
    
    rgb_image = rxr.open_rasterio(raw_fn,masked = True, drop=True)
    rgb_image = rgb_image.rio.clip(basin.geometry, basin.crs)
    _, _, _, _, rgb_image = calc_rgb(rgb_image)

    
    model_image = rxr.open_rasterio(model_fn, masked=True)
    model_image.values = np.where(np.isnan(model_image.values), 0, model_image.values)
    model_image = model_image.rio.clip(basin.geometry, basin.crs)
    
    aso = rxr.open_rasterio(aso_fn, masked=True)

### Creating netcdf of processed snow maps

In [ ]:
# Processed snow maps
p_dir = f'/data0/images/planet/emma/planet/rerun/BUDD/V5/'
p_tifs = glob.glob(p_dir + '*.tif')
# Post processed snow maps
pp_nc = '/data0/images/planet/emma/planet/pp_rerun/BUDD_V5_2023_10_10_NDVI_QAQC.nc'
pp = xr.open_dataset(pp_nc)

PP_times = pp.time.values
PP_times

dates_list = []
data_list = []
for fcount, file in enumerate(sorted(p_tifs)):
    date = os.path.basename(file).split('_')[0]
    date = datetime.strptime(date, '%Y%m%d')
    date = np.datetime64(date)
    
    if date in PP_times:
        print(date)
        dates_list.append(date)
        
        data = rxr.open_rasterio(file, drop = True, masked = True)
        data.values = np.where(0,np.isnan(data.values), data.values)
        clipped_data = data.rio.clip(basin.geometry.values, basin.crs)
        clipped_data = clipped_data.rio.clip(rgi_mask.geometry.values, invert= True)
        clipped_data = clipped_data.rio.clip(wbd_mask.geometry.values, invert= True)
        data_list.append(clipped_data.squeeze().values)
dates = np.array(dates_list)
dates = pd.to_datetime(dates, format='%Y%m%d')
stacked_data = np.stack(data_list, axis = 0).astype(float)
xr_data = xr.DataArray(stacked_data, dims=['time', 'y', 'x'], coords={'time': dates, 'y': np.arange(stacked_data.shape[1]), 'x': np.arange(stacked_data.shape[2])})
xr_data.to_netcdf(f'BUDD_V5_2023_model_output.nc',format='NETCDF4')

In [ ]:
name = 'BUDD'
model = 'V5'
year = '2020'
# Basin shapefile
BS = f'/home/etboud/projects/data/basins/{name}/{name}_4326.geojson'
basin = gpd.read_file(BS)
basin = basin.to_crs('EPSG:32611') 

rgi = '/home/etboud/projects/data/RGI/02_rgi60_WesternCanadaUS/02_rgi60_WesternCanadaUS.shp'
rgi_mask = gpd.read_file(rgi).to_crs('EPSG:32611')
wbd = '/home/etboud/projects/data/masks/NHDWaterbody.shp'
wbd_mask = gpd.read_file(wbd).to_crs('EPSG:32611')
# Raw planet images

In [ ]:
# Processed snow maps
# p_dir = f'/data0/images/planet/emma/planet/rerun/BUDD/V5/'
p_dir = '/home/etboud/projects/data/temp/BUDD/V9/'
p_tifs = glob.glob(p_dir + '*.tif')
# Post processed snow maps
pp_nc = '/home/etboud/projects/snow_mapping/Emma/BUDD_V5PP_2020_3_50_NDVI_QAQC_2.nc'
pp = xr.open_dataset(pp_nc)

PP_times = pp.time.values
PP_times

dates_list = []
data_list = []
for fcount, file in enumerate(sorted(p_tifs)):
    date = os.path.basename(file).split('_')[0]
    date = datetime.strptime(date, '%Y%m%d')
    date = np.datetime64(date)
    
    if date in PP_times:
        print(date)
        dates_list.append(date)
        
        data = rxr.open_rasterio(file, drop = True, masked = True)
        data.values = np.where(0,np.isnan(data.values), data.values)
        clipped_data = data.rio.clip(basin.geometry.values, basin.crs)
        clipped_data = clipped_data.rio.clip(rgi_mask.geometry.values, invert= True)
        clipped_data = clipped_data.rio.clip(wbd_mask.geometry.values, invert= True)
        data_list.append(clipped_data.squeeze().values)
dates = np.array(dates_list)
dates = pd.to_datetime(dates, format='%Y%m%d')
stacked_data = np.stack(data_list, axis = 0).astype(float)
xr_data = xr.DataArray(stacked_data, dims=['time', 'y', 'x'], coords={'time': dates, 'y': np.arange(stacked_data.shape[1]), 'x': np.arange(stacked_data.shape[2])})
xr_data.to_netcdf(f'BUDD_V_2020_model_output.nc',format='NETCDF4')